In [2]:
import configparser
import pymongo
from pymongo import MongoClient
from datetime import datetime
import pickle
import re
import gensim

import sys
# set python syspath to point out location of our self-writing module
sys.path.append("/home/ponshane/work_dir/CLTM/src/codebase/")

from helper import *

### init and read config
config = configparser.ConfigParser()
config.read('../config.ini')

MongoDB = config["MLDoc"]["Database"]
MongoUser = config["MLDoc"]["User"]
MongoPW = config["MLDoc"]["PW"]

###連接MONGO
uri = "mongodb://" + MongoUser + ":" + MongoPW + "@140.117.69.70:30241/" + MongoDB + "?authMechanism=SCRAM-SHA-1"

client = MongoClient(uri)
db = client.MLDoc

# For Chinese

In [7]:
### 檢查單字是否都是 英文字/數字 http://hzy3774.iteye.com/blog/2359032
def judge_pure_english(keyword):  
    return all(ord(c) < 128 for c in keyword)

def extract_selected_pos_word(sentence):
    # 挑取 #N, #V 詞性開頭的字
    indices = [i for i, s in enumerate(sentence) if "#N" in s or "#V" in s]
    select_words = [sentence[index] for index in indices]
    
    # 清除詞性標籤
    words = [re.search('(.*)#', word).group(1) for word in select_words]
    
    # 過濾單詞
    words = [word for word in words if len(word) >= 2]
    
    # 過濾英文字
    words = [word for word in words if judge_pure_english(word) == False]
    return(words)

In [6]:
# convert your date string to datetime object
target_collection = db.Chinese
num = target_collection.count({"chi_nlp_process": {"$exists": True}})
print("Number of Chinese Documents: %d" % num)

Number of Chinese Documents: 24533


In [ ]:
start_time = datetime.now()

query_documents = target_collection.find({"chi_nlp_process": {"$exists": True}}, no_cursor_timeout=True)

sentence = list()
index = 0

for each_document in query_documents:
    for each_sentence in each_document["chi_result"]["pos"]:
        sentence.append(extract_selected_pos_word(each_sentence))
    
    index += 1
    if(index % 1000 ==0):
        print("Already process %d documents" % index)

query_documents.close()

time_elapsed = datetime.now() - start_time 

print('Time elapsed (hh:mm:ss.ms) {}'.format(time_elapsed))
# Time elapsed (hh:mm:ss.ms) 0:00:18.737113

In [11]:
sentence[:5]

[['路透社',
  '香港',
  '中国',
  '电子',
  '彩管',
  '生产商',
  '深圳',
  '赛格',
  '股份',
  '有限公司',
  '表示',
  '今年',
  '上半年',
  '产量',
  '增加',
  '预计',
  '上半年',
  '利润',
  '增幅'],
 ['年报', '显示', '实现', '利润'],
 ['亿元', '人民币', '中期', '实现', '万元'],
 ['公司',
  '人士',
  '香港',
  '中国',
  '快讯',
  '表示',
  '赛格',
  '公司',
  '实施',
  '每股',
  '送股',
  '分红',
  '方案',
  '上半年',
  '利润',
  '保持'],
 []]

In [ ]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',\
    level=logging.INFO)

model = gensim.models.Word2Vec(sentence, size=100, window=5,
                               min_count=15, workers=4, negative =10)

In [14]:
model.wv.most_similar("利润")

[('净利润', 0.8554553985595703),
 ('多万元', 0.8341077566146851),
 ('盈利', 0.8136324882507324),
 ('利润总额', 0.7878793478012085),
 ('收入', 0.7827474474906921),
 ('销售收入', 0.7672287225723267),
 ('投资收益', 0.7518516182899475),
 ('亿万元', 0.745165228843689),
 ('扭亏', 0.7345239520072937),
 ('利税', 0.724858283996582)]

In [16]:
# expoert to embedding file
file_path = "../out/MLDoc/Chinese_embeddings.tsv"
out = open(file_path,'w')

for each_word in model.wv.index2word:
    out.write('\t'.join(map(str, model[each_word])) + "\n")

out.close()

# expoert to metadata file
file_path = "../out/MLDoc/Chinese_metadata.tsv"
out = open(file_path,'w')
for each_word in model.wv.index2word:
    out.write(each_word+"\n")

out.close()

model.save("../out/MLDoc/MLDoc-chinese-word2vec_NV_s100w5m15n10.vec")

/home/ponshane/anaconda3/envs/cross-lingual/lib/python3.6/site-packages/ipykernel/__main__.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
2018-11-22 10:30:07,962 : INFO : saving Word2Vec object under ../out/MLDoc/MLDoc-chinese-word2vec_NV_s100w5m15n10.vec, separately None
2018-11-22 10:30:07,963 : INFO : not storing attribute vectors_norm
2018-11-22 10:30:07,964 : INFO : not storing attribute cum_table
2018-11-22 10:30:08,015 : INFO : saved ../out/MLDoc/MLDoc-chinese-word2vec_NV_s100w5m15n10.vec


# For English

In [ ]:
start_time = datetime.now()

target_collection = db.English
docs = target_collection.find({"nlp_process": {"$exists": True}}, no_cursor_timeout=True)

index = 0
sentence_list = []
id_mongo_dictionary = {}

for doc in docs:

    for each_sentence in doc["nested_token_list"]:
        tokens_from_each_sentence = project_function_for_every_document(each_sentence, want_stop=False,
                                                                        want_alpha=True, want_lemma=True,
                                                                        accept_pos = ["NOUN", "VERB"],
                                                                        use_entity=False)
        sentence_list.append(tokens_from_each_sentence)

    index += 1
    if(index % 5000 ==0):
        print("Already process %d documents" % index)
        
docs.close()

time_elapsed = datetime.now() - start_time

print('Time elapsed (hh:mm:ss.ms) {}'.format(time_elapsed))
# Time elapsed (hh:mm:ss.ms) 0:15:07.259858

In [23]:
# pickle.dump( sentence_list, open( "../out/MLDoc/MLDoc_Eng_Sentences.pkl", "wb" ) )
len(sentence_list)

9592033

In [ ]:
model = gensim.models.Word2Vec(sentence_list, size=100, window=5,
                               min_count=15, workers=4, negative =10, sample = 1e-5)
# it takes about 4min

In [25]:
model.wv.most_similar("oil")

2020-08-26 04:45:59,229 : INFO : precomputing L2-norms of word weight vectors


[('crude', 0.7671653628349304),
 ('petroleum', 0.750298261642456),
 ('gas', 0.7449700832366943),
 ('barrel', 0.7310149073600769),
 ('liquefy', 0.715983510017395),
 ('kilolitre', 0.7007793188095093),
 ('refinery', 0.6914912462234497),
 ('refiner', 0.6849839687347412),
 ('bpd', 0.679957389831543),
 ('heating', 0.6708714962005615)]

In [26]:
# expoert to embedding file
file_path = "../out/MLDoc/English_embeddings.tsv"
out = open(file_path,'w')

for each_word in model.wv.index2word:
    out.write('\t'.join(map(str, model[each_word])) + "\n")

out.close()

# expoert to metadata file
file_path = "../out/MLDoc/English_metadata.tsv"
out = open(file_path,'w')
for each_word in model.wv.index2word:
    out.write(each_word+"\n")

out.close()

model.save("../out/MLDoc/MLDoc-english-word2vec_NV_s100w5m15n10sam1e-5.vec")

/home/ponshane/anaconda3/envs/cross-lingual/lib/python3.6/site-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
2020-08-26 04:46:17,152 : INFO : saving Word2Vec object under ../out/MLDoc/MLDoc-english-word2vec_NV_s100w5m15n10sam1e-5.vec, separately None
2020-08-26 04:46:17,152 : INFO : not storing attribute vectors_norm
2020-08-26 04:46:17,153 : INFO : not storing attribute cum_table
2020-08-26 04:46:17,287 : INFO : saved ../out/MLDoc/MLDoc-english-word2vec_NV_s100w5m15n10sam1e-5.vec


# For Japanese
Notice that we don't use sentence segmentation here for the limitation of Mecab.

In [3]:
def extract_selected_pos_word_from_jap(doc):
    # 挑取 #N, #V 詞性開頭的字
    indices = [i for i, s in enumerate(doc) if "#名詞" in s or "#動詞" in s]
    select_words = [doc[index] for index in indices]
    
    # 清除詞性標籤
    words = [re.search('(.*)#', word).group(1) for word in select_words]
    
    # 過濾單詞
    words = [word for word in words if len(word) >= 2]
    
    # 過濾英文字
    words = [word for word in words if judge_pure_english(word) == False]
    return(words)

In [5]:
# convert your date string to datetime object
target_collection = db.Japanese
num = target_collection.count_documents({"jap_nlp_process": {"$exists": True}})
print("Number of Chinese Documents: %d" % num)

Number of Chinese Documents: 58599


In [8]:
start_time = datetime.now()

query_documents = target_collection.find({"jap_nlp_process": {"$exists": True}}, no_cursor_timeout=True)

jap_sentences = list()
index = 0

for each_document in query_documents:
    jap_sentences.append(extract_selected_pos_word_from_jap(each_document["jap_result"]["pos"]))
    
    index += 1
    if(index % 1000 ==0):
        print("Already process %d documents" % index)

query_documents.close()

time_elapsed = datetime.now() - start_time 

print('Time elapsed (hh:mm:ss.ms) {}'.format(time_elapsed))
# Time elapsed (hh:mm:ss.ms) 0:00:18.737113

Already process 1000 documents
Already process 2000 documents
Already process 3000 documents
Already process 4000 documents
Already process 5000 documents
Already process 6000 documents
Already process 7000 documents
Already process 8000 documents
Already process 9000 documents
Already process 10000 documents
Already process 11000 documents
Already process 12000 documents
Already process 13000 documents
Already process 14000 documents
Already process 15000 documents
Already process 16000 documents
Already process 17000 documents
Already process 18000 documents
Already process 19000 documents
Already process 20000 documents
Already process 21000 documents
Already process 22000 documents
Already process 23000 documents
Already process 24000 documents
Already process 25000 documents
Already process 26000 documents
Already process 27000 documents
Already process 28000 documents
Already process 29000 documents
Already process 30000 documents
Already process 31000 documents
Already process 3

In [ ]:
jap_sentences[:5]

In [16]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',\
    level=logging.INFO)

In [17]:
model = gensim.models.Word2Vec(jap_sentences, size=100, window=5,
                               min_count=15, workers=4, negative =10, sample=1e-4)
# it takes about 4min

2020-08-26 04:06:22,295 : INFO : collecting all words and their counts
2020-08-26 04:06:22,296 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2020-08-26 04:06:22,381 : INFO : PROGRESS: at sentence #10000, processed 911461 words, keeping 28743 word types
2020-08-26 04:06:22,465 : INFO : PROGRESS: at sentence #20000, processed 1818927 words, keeping 39075 word types
2020-08-26 04:06:22,552 : INFO : PROGRESS: at sentence #30000, processed 2723166 words, keeping 47590 word types
2020-08-26 04:06:22,639 : INFO : PROGRESS: at sentence #40000, processed 3624877 words, keeping 53701 word types
2020-08-26 04:06:22,723 : INFO : PROGRESS: at sentence #50000, processed 4505889 words, keeping 58801 word types
2020-08-26 04:06:22,798 : INFO : collected 63302 word types from a corpus of 5277657 raw words and 58599 sentences
2020-08-26 04:06:22,798 : INFO : Loading a fresh vocabulary
2020-08-26 04:06:22,930 : INFO : min_count=15 retains 12894 unique words (20% of original 6

In [19]:
model.wv.most_similar("石油")

[('原油', 0.7877740859985352),
 ('石油製品', 0.7258394956588745),
 ('原油価格', 0.7218372225761414),
 ('オイル', 0.7171489596366882),
 ('ガス', 0.7155888080596924),
 ('ナイジェリア', 0.7009726762771606),
 ('天然ガス', 0.6996159553527832),
 ('産油', 0.6895568370819092),
 ('イラク', 0.6825350522994995),
 ('産油国', 0.6799705028533936)]

In [20]:
# expoert to embedding file
file_path = "../out/MLDoc/Japanese_embeddings.tsv"
out = open(file_path,'w')

for each_word in model.wv.index2word:
    out.write('\t'.join(map(str, model[each_word])) + "\n")

out.close()

# expoert to metadata file
file_path = "../out/MLDoc/Japanese_metadata.tsv"
out = open(file_path,'w')
for each_word in model.wv.index2word:
    out.write(each_word+"\n")

out.close()

model.save("../out/MLDoc/MLDoc-japanese-word2vec_NV_s100w5m15n10sam1e-4.vec")

/home/ponshane/anaconda3/envs/cross-lingual/lib/python3.6/site-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
2020-08-26 04:09:17,544 : INFO : saving Word2Vec object under ../out/MLDoc/MLDoc-japanese-word2vec_NV_s100w5m15n10sam1e-4.vec, separately None
2020-08-26 04:09:17,544 : INFO : not storing attribute vectors_norm
2020-08-26 04:09:17,545 : INFO : not storing attribute cum_table
2020-08-26 04:09:17,635 : INFO : saved ../out/MLDoc/MLDoc-japanese-word2vec_NV_s100w5m15n10sam1e-4.vec
